## Test Data Prediction
This notebook will be used to open test.csv file and use designated model to predict the test data for kaggle submission.

In [10]:
%%bash
git branch

  CV_check
  EDA1/wip
  cost1/feat1/wip
  cost1/feat2/wip
  cost1/wip
  feat1/score1/wip
  feat1/wip
  feat2/score1/wip
  feat2/wip
  feat3/wip
  feat4/wip
  feat5/wip
* feat6/wip
  feat7(EDA1)/wip
  master


In [8]:
%%bash
git checkout 'feat7(EDA1)/wip'
git fetch

error: pathspec 'feat7(EDA)/wip' did not match any file(s) known to git.


In [16]:
%%bash
git add 'Test_data.ipynb'
git commit -m 'Dummy push'
git push --set-upstream origin feat2/wip

[feat6/wip 29a0667] Created submission file
 1 file changed, 1192 insertions(+), 957 deletions(-)
 rewrite Test_data.ipynb (68%)
Branch feat2/wip set up to track remote branch feat2/wip from origin.


Everything up-to-date


In [3]:
model_ver = 'feat6.sav'
submission_name = 'feat6_test.csv'

In [4]:
%matplotlib inline 
import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn.model_selection
import pickle

/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Load data from csv file then prepare the data before inputting into the trained model.

In [5]:
test_ex = pd.read_csv('gender_submission.csv')
test_ex.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [6]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
#train = train.drop('Survived',axis=1)
#Concate train and test to avoid mismatch features in the testing phase
#train_objs_num = len(train)
#dataset = pd.concat(objs=[train, test], axis=0)
#dataset = pd.get_dummies(dataset)
#train = dataset[:train_objs_num].copy()
#test = dataset[train_objs_num:].copy()
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Data transform train and test to avoid feature mismatched.

In [8]:
#Make sure the same thins is applied to train and test data.
train = train.drop('PassengerId',axis=1)
train = train.drop('Name',axis=1)
train = train.drop('Cabin',axis=1)
test = test.drop('PassengerId',axis=1)
test = test.drop('Name',axis=1)
test = test.drop('Cabin',axis=1)

num_attri = ['Age','SibSp','Parch','Fare']
#cat_attri = [column for column in test if (column not in num_attri and column != 'Survived')]
cat_attri = [column for column in train if (column not in num_attri and column != 'Survived')] 

for column in (cat_attri): #Create dummy for X
    #Lets just keep every categorical attributes for plotting
    tempDF = pd.get_dummies(train[column],prefix=column)
    train.drop(column,axis=1,inplace=True)
    train = train.join(tempDF)

for column in (cat_attri): #Create dummy for test
    #Lets just keep every categorical attributes for plotting
    tempDF = pd.get_dummies(test[column],prefix=column)
    test.drop(column,axis=1,inplace=True)
    test = test.join(tempDF)

#Concat X(train data) with X from test data to avoid mismatch feature
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
#dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num].copy()
test = dataset[train_objs_num:].copy()

#Drop Survived column at the same step in training file
test = test.drop('Survived',axis=1)

In [9]:
cat_attri

['Pclass', 'Sex', 'Ticket', 'Embarked']

In [10]:
test.head()

,Age,Embarked_C,Embarked_Q,Embarked_S,Fare,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,...,Ticket_W./C. 14258,Ticket_W./C. 14260,Ticket_W./C. 14263,Ticket_W./C. 14266,Ticket_W./C. 6607,Ticket_W./C. 6608,Ticket_W./C. 6609,Ticket_W.E.P. 5734,Ticket_W/C 14208,Ticket_WE/P 5735
0,34.5,0,1,0,7.8292,0,0,0,1,0,...,NaN,0.0,NaN,0.0,0,0,NaN,0,NaN,NaN
1,47.0,0,0,1,7.0000,0,0,0,1,1,...,NaN,0.0,NaN,0.0,0,0,NaN,0,NaN,NaN
2,62.0,0,1,0,9.6875,0,0,1,0,0,...,NaN,0.0,NaN,0.0,0,0,NaN,0,NaN,NaN
3,27.0,0,0,1,8.6625,0,0,0,1,0,...,NaN,0.0,NaN,0.0,0,0,NaN,0,NaN,NaN
4,22.0,0,0,1,12.2875,1,0,0,1,1,...,NaN,0.0,NaN,0.0,0,0,NaN,0,NaN,NaN


Bar plot to visualize data in test set

In [11]:
for i,column in enumerate(cat_attri):
    #Lets just keep every categorical attributes for plotting
    plt.figure(i)
    sns.countplot(test[column])

KeyError: 'Pclass'

Use the saved model to predict the answers

In [12]:
loaded_model = pickle.load(open(model_ver, 'rb'))
Y_predict = loaded_model.predict(test)
Y_predict

/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,
        0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
        1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,
        1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
        1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0

In [13]:
test_temp = pd.read_csv('Test.csv')
sub_file = pd.concat([test_temp['PassengerId'],pd.DataFrame(Y_predict)],axis=1)
sub_file.columns =['PassengerId','Survived']
sub_file = sub_file.astype(int)
#pd.options.display.float_format = '{:,.0f}'.format

In [14]:
sub_file.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [15]:
sub_file.to_csv(path_or_buf = submission_name,index=False)